In [15]:
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn.utils.rnn as rnn_utils
import numpy as np
import pandas as pd
import random
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from torch.utils.data import random_split

In [16]:
# set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)


In [17]:
# Load data
df_full = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep='\t', compression='zip')
# df_full.info()

In [18]:
# padding function for each movie review sentence
def create_padding(sentence):
    words = sentence.split()
    words = words[:10] if len(words) > 10 else words + ["<PAD>"] * (10 - len(words))
    words = ' '.join(words)
    return words



In [19]:
# Relevant columns only

data = df_full.drop(['SentenceId'], axis=1)
data = data.iloc[:5000, :]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PhraseId   5000 non-null   int64 
 1   Phrase     5000 non-null   object
 2   Sentiment  5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [20]:
# Remove punctuations

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data["Phrase"] = data["Phrase"].apply(lambda text: remove_punctuation(text))

# Remove STOPWORDS

", ".join(stopwords.words('english'))

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data["Phrase"] = data["Phrase"].apply(lambda text: remove_stopwords(text))

# Remove most common words

cnt = Counter()
for text in data["Phrase"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

data["Phrase"] = data["Phrase"].apply(lambda text: remove_freqwords(text))

# # Remove Stemming 

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

# data["Phrase"] = data["Phrase"].apply(lambda text: stem_words(text))

# # Lemmatisation

# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()
# wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
# def lemmatize_words(text):
#     pos_tagged_text = nltk.pos_tag(text.split())
#     return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# data["Phrase"] = data["Phrase"].apply(lambda text: lemmatize_words(text))

In [21]:
# apply the function to the dataframe column 'Phrase'

# data = data[data['Phrase'].apply(lambda x: len(x.split()) >= 3)]

data['Phrase'] = data['Phrase'].apply(lambda x: create_padding(x))


In [22]:
# Tokenize and pad sequences
vocab = set(" ".join(df_full["Phrase"]).split())
vocab.add("<PAD>")
word_to_ix = {word: i+1 for i, word in enumerate(vocab)}

In [23]:
# collate function to provide equal length of tokens in each row of the batch
def collate_fn(batch):
    # Assuming each element of batch is a sequence of tensors
    # Pad sequences to the same length
    x_batch, y_batch = zip(*batch)
    x_batch = rnn_utils.pad_sequence(x_batch, batch_first=True)
    return x_batch, y_batch

In [24]:
# Define dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.num_classes = len(set(data["Sentiment"]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index]["Phrase"]
        y = self.data.iloc[index]["Sentiment"]
        x_tokenized = self.tokenizer(x)
#         print(type(x_tokenized))
        x_tokenized_len = len(x_tokenized)
        x_tokenized_ids = [word_to_ix[word] if word in word_to_ix else word_to_ix['<PAD>'] for word in x_tokenized]
        x_tokenized_tensor = torch.tensor(x_tokenized_ids)
#         print(f'x_tokenized: {x_tokenized_tensor} and y: {y}')
        return x_tokenized_tensor, y

In [25]:
# Define model architecture
class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(device)
        c0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(device)
        out, _ = self.rnn(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [26]:
# Define model parameters
input_size = len(vocab) + 1
hidden_size = 16
output_size = 5
num_layers = 3

In [27]:
# Rebalancing an unbalanced dataset
# Find class weights
class_counts = data['Sentiment'].value_counts()
print(data['Sentiment'].values)
print(class_counts)
class_weights = 1 / torch.tensor(class_counts, dtype=torch.float)
print(class_weights)
# Compute weights for each data point
weights = class_weights[data['Sentiment'].values]
print(weights[4])
# Create a sampler for weighted random sampling
sampler = WeightedRandomSampler(weights, len(weights), replacement=False)


[1 2 2 ... 3 2 2]
2    2919
3     898
1     777
4     251
0     155
Name: Sentiment, dtype: int64
tensor([0.0003, 0.0011, 0.0013, 0.0040, 0.0065])
tensor(0.0013)


In [28]:
def validate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch = torch.stack(tuple(x_batch)).to(device)
            y_batch = torch.LongTensor(y_batch).to(device)
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            running_loss += loss.item() * x_batch.size(0)
    return running_loss / len(data_loader.dataset)

# Train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentRNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
batch_size = 64
num_epochs = 250
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
dataset = SentimentDataset(data, tokenizer)

# Split dataset into train and validation sets
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(len(train_dataset))

# Loader with weighted sampling enabled
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=None, collate_fn=collate_fn, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=None, collate_fn=collate_fn, drop_last=True)

# variables to store best loss and best model
best_val_loss = float('inf')
best_model_path = '/kaggle/working/best_model.pth'

# Train loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0    
    for x_batch, y_batch in train_loader:
        x_batch = torch.stack(tuple(x_batch)).to(device)
        y_batch = torch.tensor(y_batch).to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x_batch.size(0)
    epoch_loss = running_loss / len(dataset)
    # Validation code here
    val_loss = validate(model, val_loader, criterion)
    
    # Save best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_path = 'best_model.pth'
        torch.save(model.state_dict(), best_model_path)
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

4000
Epoch 1/250, Loss: 0.9458
Epoch 2/250, Loss: 0.8592
Epoch 3/250, Loss: 0.8193
Epoch 4/250, Loss: 0.7788
Epoch 5/250, Loss: 0.7536
Epoch 6/250, Loss: 0.7432
Epoch 7/250, Loss: 0.7208
Epoch 8/250, Loss: 0.6949
Epoch 9/250, Loss: 0.6673
Epoch 10/250, Loss: 0.6526
Epoch 11/250, Loss: 0.6388
Epoch 12/250, Loss: 0.6438
Epoch 13/250, Loss: 0.6321
Epoch 14/250, Loss: 0.5944
Epoch 15/250, Loss: 0.5759
Epoch 16/250, Loss: 0.5541
Epoch 17/250, Loss: 0.5287
Epoch 18/250, Loss: 0.5111
Epoch 19/250, Loss: 0.4837
Epoch 20/250, Loss: 0.4746
Epoch 21/250, Loss: 0.4559
Epoch 22/250, Loss: 0.4464
Epoch 23/250, Loss: 0.4407
Epoch 24/250, Loss: 0.4379
Epoch 25/250, Loss: 0.4319
Epoch 26/250, Loss: 0.4375
Epoch 27/250, Loss: 0.4286
Epoch 28/250, Loss: 0.4260
Epoch 29/250, Loss: 0.4147
Epoch 30/250, Loss: 0.4156
Epoch 31/250, Loss: 0.4061
Epoch 32/250, Loss: 0.4065
Epoch 33/250, Loss: 0.4005
Epoch 34/250, Loss: 0.3901
Epoch 35/250, Loss: 0.3840
Epoch 36/250, Loss: 0.3887
Epoch 37/250, Loss: 0.3846
Epoch

In [29]:
# Load best model:
best_model = SentimentRNN(input_size, hidden_size, output_size, num_layers).to(device)
best_model.load_state_dict(torch.load(best_model_path))

# Evaluate model

best_model.eval() # Set model to evaluation mode
test_sentence = "This movie was terrible. I can not stand this move!"
test_sequence = [word_to_ix[word] if word in word_to_ix else 0 for word in test_sentence.split()[:10]]
test_sequence += [0] * (10 - len(test_sequence))
test_sequence = torch.tensor(test_sequence, dtype=torch.long).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(test_sequence)
    prediction = torch.argmax(output, dim=1).item()
print(f"Test Sentence: {test_sentence}")
print(f"Prediction: {prediction}")

Test Sentence: This movie was terrible. I can not stand this move!
Prediction: 3


In [30]:
# test file and predictions
# Load data
df_test = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep='\t', compression='zip')
# Relevant columns only
data_test = df_test.drop(['SentenceId'], axis=1)

# Remove punctuations

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data_test["Phrase"] = data_test["Phrase"].apply(lambda text: remove_punctuation(text))

# Remove STOPWORDS

", ".join(stopwords.words('english'))

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data_test["Phrase"] = data_test["Phrase"].apply(lambda text: remove_stopwords(text))

# Remove most common words

cnt = Counter()
for text in data_test["Phrase"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

data_test["Phrase"] = data_test["Phrase"].apply(lambda text: remove_freqwords(text))

# # Remove Stemming 

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

# data_test["Phrase"] = data_test["Phrase"].apply(lambda text: stem_words(text))

# # Lemmatisation

# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()
# wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
# def lemmatize_words(text):
#     pos_tagged_text = nltk.pos_tag(text.split())
#     return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# data_test["Phrase"] = data_test["Phrase"].apply(lambda text: lemmatize_words(text))

data_test.info()
# apply the function to the dataframe column 'Phrase'
# data_test = data_test.iloc[:10000, :]
data_test['Phrase'] = data_test['Phrase'].apply(lambda x: create_padding(x))

# feed into the evaluation model
best_model.eval() # Set model to evaluation mode
y_pred_submission = []
for sentence_test in data_test['Phrase']:
    sentence_test = [word_to_ix[word] if word in word_to_ix else 0 for word in sentence_test.split()[:10]]
    sentence_test += [0] * (10 - len(sentence_test))
    sentence_test = torch.tensor(sentence_test, dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(sentence_test)
        prediction = torch.argmax(output, dim=1).item()
        y_pred_submission.append(prediction)
d = {'PhraseId': data_test['PhraseId'], 'Sentiment': prediction} 
submission = pd.DataFrame(data=d)
submission

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PhraseId  66292 non-null  int64 
 1   Phrase    66292 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


,PhraseId,Sentiment
0,156061,0
1,156062,0
2,156063,0
3,156064,0
4,156065,0
...,...,...
66287,222348,0
66288,222349,0
66289,222350,0
66290,222351,0


In [31]:
submission.to_csv('/kaggle/working/submission.csv', index=False, header=True)